# 1.Term Frequency(TF)

In [1]:
import math
import pandas as pd
import numpy as np

In [2]:
#documents
doc1="happy happy happy happy happy  cat cat cat cat cat cat cat cat cat cat cat cat "
doc2="cat cat cat cat happy happy happy "
#query striing
query="happy cat"

In [3]:
#text preprocessing steps are ignored as the objective of this kernal is to expalin and develop TF-IDF and cosine similarity from scratch

In [4]:
#term - frequency : word occurences in a document

def compute_tf(docs_list):
    for doc in docs_list:
        doc1_lst=doc.split(" ")
        
        wordDict_1=dict.fromkeys(set(doc1_lst),0)
        
        for token in doc1_lst:
            wordDict_1[token]+=1
        df=pd.DataFrame([wordDict_1])
        idx=0
        
        new_col=["Term Frequency"]
        
        df.insert(loc=idx,column="Document",value=new_col)
        
        print(df)
        

compute_tf([doc1,doc2])
        
    

         Document     cat  happy
0  Term Frequency  2   12      5
         Document     cat  happy
0  Term Frequency  1    4      3


in reality each document will be of different size. On a large document the frequency of the terms will be much higher than the smaller ones . Hence we need to normaluze the document based on its size

A simple  trick is to divide the term frequency by the total number of terms 

for example in document 1 the term "game" occures 2 times. the total number of terms in the document is 10. Hence the normalized term frequency is 2/10=0.2


Given below are the normalized term frequency for all documents 

In [5]:
#normalized term frequency
def termFrequency(term,document):
    normalizeDocumnet =document.lower().split()
    return normalizeDocumnet.count(term.lower())/float(len(normalizeDocumnet))
    
    
    
def compute_normalizedTF(documents):
    tf_doc=[]
    
    for txt in documents:
        sentence =txt.split()
        norm_tf=dict.fromkeys(set(sentence),0)
        
        for word in sentence:
            norm_tf[word]=termFrequency(word,txt)
            
        tf_doc.append(norm_tf)
        
        df=pd.DataFrame([norm_tf])
        
        idx=0
        new_col=["Normallized TF"]
        
        df.insert(loc=idx,column="Document",value=new_col)
        print(df)
        
    return tf_doc

tf_doc=compute_normalizedTF([doc1,doc2])
tf_doc

         Document       cat     happy
0  Normallized TF  0.705882  0.294118
         Document       cat     happy
0  Normallized TF  0.571429  0.428571


[{'cat': 0.7058823529411765, 'happy': 0.29411764705882354},
 {'cat': 0.5714285714285714, 'happy': 0.42857142857142855}]

# 2) IDF

In [6]:
def IDF(term,allDocuments):
    numDocumentsWithThisTerm=0
    
    for doc in range (0,len(allDocuments)):
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm=numDocumentsWithThisTerm+1
            
    
    if numDocumentsWithThisTerm>0:
        return 1.0+math.log(float(len(allDocuments))/numDocumentsWithThisTerm)
    else:
        return 1.0

    
    
def compute_idf(documents):
    idf_dict={}
    for doc in documents :
        sentence=doc.split()
        for word in sentence:
            idf_dict[word]=IDF(word,documents)
            
    return idf_dict


idf_dict = compute_idf([doc1,doc2])

compute_idf([doc1,doc2])

{'happy': 1.0, 'cat': 1.0}

# 3) TF-IDF

Remember we are trying to find out relevant documents for the query "life learning"

for each term in the query multiply its normalized term frequency with its IDF on each document

in document1 for the term "life " the normalized term frequency is 0.1 and its IDF is 1.4054651081081644

Multiplying them together we get (0.1 * 1.4054651081081644)

Given below is TF * IDF calculations for "life" and "learning" in all the documents

In [7]:
#tf-idf score across all docs for the query string ("life learning")
def compute_tfidf_with_alldocs(documents,query):
    tf_idf=[]
    index=0
    
    query_tokens=query=query.split()
    df=pd.DataFrame(columns=["doc ID"]+query_tokens)
    
    for doc in documents:
        df["doc ID"]=np.arange(0,len(documents))
        doc_num=tf_doc[index]
        
        sentence=doc.split()
        
        for word in sentence:
            for text in query_tokens:
                if(text==word):
                    idx=sentence.index(word)
                    tf_idf_score=doc_num[word]*idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index,df.columns.get_loc(word)]=tf_idf_score
                    
        index+=1
        
    df.fillna(0,axis=1,inplace=True)
    
    return tf_idf,df



documents=[doc1,doc2]
tf_idf,df=compute_tfidf_with_alldocs(documents,query)

print(df)

   doc ID     happy       cat
0     0.0  0.294118  0.705882
1     1.0  0.428571  0.571429


# 4) Vector space models and representation - cosine similarity 

In [8]:
#normalized TF for the query string ("life learning")
def compute_query_tf(query):
    query_norm_tf={}
    tokens=query.split()
    for word in tokens:
        query_norm_tf[word]=termFrequency(word,query)
    return query_norm_tf

query_norm_tf=compute_query_tf(query)
print(query_norm_tf)

{'happy': 0.5, 'cat': 0.5}


In [9]:
#idf score for the query string("life learning")
def compute_query_idf(query):
    idf_dict_qry={}
    sentence = query.split()
    documents=[doc1,doc2]
    
    for word in sentence:
        idf_dict_qry[word]=IDF(word,documents)
        
    return idf_dict_qry

idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)


    

{'happy': 1.0, 'cat': 1.0}


In [10]:
#tf-idf score for the query string ("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry={}
    sentence=query.split()
    
    for word in sentence:
        tfidf_dict_qry[word]=query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry

tfidf_dict_qry=compute_query_tfidf(query)

print(tfidf_dict_qry)

{'happy': 0.5, 'cat': 0.5}


In [11]:
def cosine_similarity(tfidf_dict_qry,df,query,doc_num):
    
    dot_product=0
    qry_mod=0
    doc_mod=0
    tokens=query.split()
    
    for keyword in tokens:
        dot_product+=tfidf_dict_qry[keyword] * df[keyword][df['doc ID'] ==doc_num]
        
        #||Query||
        qry_mod+=tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        
        #||Document||
        doc_mod+=df[keyword][df['doc ID']==doc_num] * df[keyword][df['doc ID']==doc_num]
        
    qry_mod = np.sqrt(qry_mod)
    doc_mod=np.sqrt(doc_mod)
    
    #implement formular
    denominator = qry_mod * doc_mod
    cos_sin = dot_product/denominator
    
    return cos_sin





In [16]:
cosine_similarity(tfidf_dict_qry,df,query,0)

0    0.924678
Name: happy, dtype: float64

In [12]:
from collections.abc import Iterable

In [13]:
def flatten(lis):
    for item in lis:
        if isinstance(item,Iterable) and not isinstance(item,str):
            for x in flatten(item):
                yield x
                
        else:
            yield item

In [14]:
def rank_similarity_docs(data):
    cos_sin = []
    for doc_num in range(0,len(data)):
        cos_sin.append(cosine_similarity(tfidf_dict_qry,df,query,doc_num).tolist())
        
    return cos_sin

similarity_docs=rank_similarity_docs(documents)

doc_names=["Document1","Document2"]

print(doc_names)
print(list(flatten(similarity_docs)))

['Document1', 'Document2']
[0.9246780984747159, 0.9899494936611667]


# Method 2

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Documents
doc1 = "happy happy happy happy happy cat cat cat cat cat cat cat cat cat cat cat cat"
doc2 = "cat cat cat cat happy happy happy"
# Query string
query = "happy cat"

# Combine documents and query into a list
documents = [doc1, doc2, query]

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Compute TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(documents)

# Calculate cosine similarity between each document and the query
cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

# Print the cosine similarities
for i, similarity in enumerate(cosine_similarities[0]):
    print(f"Cosine Similarity between Query and Doc{i+1}: {similarity}")

Cosine Similarity between Query and Doc1: 0.9246780984747162
Cosine Similarity between Query and Doc2: 0.9899494936611666
